In [ ]:
!pip install pdfplumber pinecone

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.4/421.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 57.7 MB/s eta 0:00:00


In [ ]:
import pdfplumber
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone
import os

In [ ]:
pc = Pinecone(api_key="pcsk_P5Pfd_LoH3ryXtGWHBbmVFjSRhsdJqxpmzBQyzkk1TefVrGd9TQdtah9Rpt6i1g1BENiN")
index_name = "document-search"

# Load embedding model
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

# Chunk the document
def chunk_text(text, chunk_size=500, chunk_overlap=100):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len
    )
    return text_splitter.split_text(text)

# Convert to embeddings and store in Pinecone
def store_embeddings(text_chunks):
    index = pc.Index(index_name)  # Connect to the Pinecone index
    embeddings = model.encode(text_chunks)  # Convert to embeddings

    # Store in Pinecone
    for i, (chunk, embedding) in enumerate(zip(text_chunks, embeddings)):
        index.upsert(vectors=[(str(i), embedding.tolist(), {"chunk": chunk})])

    print("Embeddings stored successfully!")
    return index

# Load PDF, chunk text, generate embeddings, and store
pdf_text = extract_text_from_pdf("/content/financial aid course.pdf")
chunks = chunk_text(pdf_text)
index = store_embeddings(chunks)

Embeddings stored successfully!


In [ ]:
query = "How can I apply for financial assistance for this course?"
query_embedding = model.encode(query).tolist()  # Convert query to vector

# Search for the top 3 similar documents
results = index.query(vector=query_embedding, top_k=3, include_metadata=True)

if results["matches"]:
    best_match = results["matches"][0]  # Get the top result
    answer = best_match["metadata"].get("chunk", "No relevant document found")  # Extract text
else:
    answer = "No relevant document found."

print("Answer:", answer)

Answer: However, as a student, I am currently facing financial constraints that make it difficult for me
to afford the cost of the program. Despite this, I am determined to pursue my passion and
invest in my future. That's why I am reaching out to inquire about any financial aid or
scholarship opportunities that may be available to assist me in obtaining this valuable
educational experience.
I am confident that the knowledge and skill I will gain from Supervised Machine Learning:
